# 📍 Recipe: Shot Accuracy per Team

This example shows how to calculate shot accuracy - the percentage of shots that were on target - for each team in a match.

## 🧰 What You'll Learn

- How to filter for shots
- How to define custom group-level aggregations
- How to compute derived metrics like accuracy %

## Imports

In [1]:
from penaltyblog.matchflow import Flow, where_equals

## Load the Data

In [2]:
# Load events for a StatsBomb match
match_id = 22912  # Champions League Final 2018/2019

flow = Flow.statsbomb.from_github_file(match_id=match_id)

## Define our Custom Aggregation Function

In [3]:
def shot_accuracy(records):
    total = 0
    on_target = 0
    for r in records:
        total += 1
        if r.get("outcome") in ("Goal", "Saved"):
            on_target += 1
    return round((on_target / total) * 100, 1) if total else 0.0

## Calculate Shot Accuracy

In [4]:
results = (
    flow
    .filter(where_equals("type.name", "Shot"))
    .select("team.name", "shot.outcome.name")
    .rename(**{"team.name": "team", "shot.outcome.name": "outcome"})
    .group_by("team")
    .summary(total_shots="count",
        accuracy_pct=shot_accuracy,
        )
    .collect()
)

results

[{'team': 'Liverpool', 'total_shots': 14, 'accuracy_pct': 21.4},
 {'team': 'Tottenham Hotspur', 'total_shots': 16, 'accuracy_pct': 50.0}]